In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import time

In [3]:
def func(x, m1, m2, m3, m4, m5, m6):
    return m1 + m2 /(1 + np.exp(-m3 * (x-m4))) - m2/(1 + np.exp(-m5 * (x-m6)))

#注意初值
def DofittingandValidation(yData, xData):
  # Parameters, pcov = curve_fit(func, xData, yData, p0=[1000,7000,0.1,140,0.1,270], maxfev=i)#,method='trf', maxfev=1000000)
  # EVI:900,5000,0.1,140,0.1,270
  # LAI:5,40,0.1,140,0.1,270
  # NDVI:1000,7000,0.1,140,0.1,270
    start_time1 = time.time()
    for i in range(1,1000000):
    
        try:
            Parameters, pcov = curve_fit(func, xData, yData, p0=[5,40,0.1,140,0.1,270], maxfev=i)#,method='trf', maxfev=1000000)
        except RuntimeError: 
            continue
        if Parameters is not None:
            break
    totaltime=time.time()-start_time1
    totalparm=np.hstack((Parameters,[i,totaltime]))
    return totalparm

#注意时间序列
def get_python_param(data):
    timeDims=data.shape[1]
  #EVI,NDVI
  # xData=16 * (np.array(list(range(timeDims)))) + 9

  #LAI
    xData=4 * (np.array(list(range(timeDims)))) + 1
    pars=[]
    for i in range(data.shape[0]):
    # for i in range(3):
        lon_lat=data_lon_lat[i]
        yData = data[i,:]
        pars_ = DofittingandValidation(yData, xData) 
        x=np.hstack((lon_lat,pars_))
        pars.append(x)
    return np.array(pars)



In [ ]:
start_time = time.time()
dataset=pd.read_excel(r'/content/drive/My Drive/tif/data/gee-LAI-10884.xlsx',header=0)
# dataset = dataset.set_index(['lat','lon'])
dataset=np.array(dataset)
data_lon_lat=dataset[:,0:2]
data=dataset[:,2::]
param=get_python_param(data)

print("Thread pool execution in " + str(time.time() - start_time), "seconds")
writer = pd.ExcelWriter(r"/content/drive/My Drive/tif/get_param_python/LAI_10884_python_param_time_item1.xls")
pd.DataFrame(param).to_excel(writer,na_rep=0,index=False,header=['lon','lat','p0','p1','p2','p3','p4','p5','item','time'])
writer.save()
writer.close()